In [12]:
#########################################################################
#                                                                       #
#   提醒助教一下，本功課分成三個部分（基礎版、with dropout版 、K-fold版）       #  
#   每個版本訓練出來的model，因為檔名用一樣所以會有取代的問題，所以如果要檢查model #
#   要注意下，因為想乾淨一點，才這樣設計的，抱歉嘿。                            #
#                                                                       #
#########################################################################

In [13]:
import numpy as np
import os
import tensorflow as tf
from sklearn.metrics import classification_report ## 印出可視覺化的表

###### Do not modify here ###### 

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]

###### Do not modify here ###### 

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [14]:
##############################################
#                                            #
#               基礎版本                      #
#                                            #
##############################################

In [15]:
##先前設置##
#28 * 28 拉直後 = 784維 
#題目要求5層hidden layers，及每一層128neurons
#題目要求只要辨識 0 ,1 ,2 ,3 及4 ，共5個類別
in_units = 784 
h1_units = 128 
h2_units = 128
h3_units = 128
h4_units = 128
h5_units = 128
n_class = 5
batch_size = 50 # 設定50，可自由設
train_size = len(X_train1) # train資料的長度
checkpoint_dir = 'save/'  #model儲存位址
config = tf.ConfigProto(device_count = {'GPU': 1}) #指定gpu


In [16]:
x = tf.placeholder(tf.float32,[None,in_units]) ##初始化x資料型態為[None,784]
y = tf.placeholder(tf.int64, shape=(None), name="y") ## 初始化y資料型態[None]

In [17]:
#keep the scale of the input variance constant,
#so it does not explode or diminish by reaching the final layer.
he_init=tf.contrib.layers.variance_scaling_initializer(factor=2.0,
    mode='FAN_IN',
    uniform=False,
    seed=None,
    dtype=tf.float32)

In [18]:
#5層的DNN結構
#tf.layers.dense 可用來連接每層，並激勵函數則用題目要求的elu
with tf.name_scope("DNN"):
    ##hidden1 總共要learn的參數[784 , 128 ]
    hidden1 = tf.layers.dense(x, h1_units, name="hidden1", use_bias=True , kernel_initializer= he_init,
                             bias_initializer=he_init , activation=tf.nn.elu)
    
    ## hideen2 總共要learn的參數[128 , 128]
    hidden2 = tf.layers.dense(hidden1, h2_units, name="hidden2", use_bias=True,kernel_initializer= he_init,
                             bias_initializer=he_init , activation=tf.nn.elu)
    
    ## hideen3 總共要learn的參數[128 , 128]
    hidden3 = tf.layers.dense(hidden2, h3_units, name="hidden3", use_bias=True , kernel_initializer= he_init,
                             bias_initializer=he_init , activation=tf.nn.elu)
    
    ## hideen4 總共要learn的參數[128 , 128]
    hidden4 = tf.layers.dense(hidden3, h4_units, name="hidden4", use_bias=True , kernel_initializer= he_init,
                             bias_initializer=he_init , activation=tf.nn.elu)
    
    ## hideen5 總共要learn的參數[128 , 128]
    hidden5 = tf.layers.dense(hidden4, h5_units, name="hidden5", use_bias=True , kernel_initializer= he_init,
                             bias_initializer=he_init , activation=tf.nn.elu)
    ## 結合之後的tf.nn.sparse_softmax_cross_entropy_with_logits [128 , 5]
    logits = tf.layers.dense(hidden5, n_class, name="outputs")

In [19]:
## 此區域製作損失函數loss,用於更新之前的W參數
## y實際類別 , logist 預測類別
## 使用tf.nn.sparse_softmax_cross_entropy_with_logits 可以讓輸入的y_不需要先換成one-hot encoding
## 使用這個方法可以讓softmax 跟 entropy 同時進行
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(entropy, name="loss")
    
## 此區使用AdamOptimizer 的優化器
## 進行梯度優化
with tf.name_scope("train"):
    batch = tf.Variable(0)

    learning_rate = tf.train.exponential_decay(
            1e-4,  # Base learning rate.
            batch * batch_size,  # Current index into the dataset.
            train_size,  # Decay step.
            0.95,  # Decay rate.
            staircase=True)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss,global_step=batch)
    
    
#計算正確率   
with tf.name_scope('accuracy'):
    correct = tf.equal(tf.argmax(logits, 1), y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

#計算precision 因返回會有兩個值，只取後者
with tf.name_scope("precision"):
    _,precision = tf.metrics.precision(predictions = tf.argmax(logits,1)
                                       , labels= y)
#計算recall 因返回會有兩個值，只取後者
with tf.name_scope('recall'):
    _,recall = tf.metrics.recall(predictions = tf.argmax(logits,1)
                                 , labels= y)


In [20]:
# Params for Train
training_epochs = 100 # 看你要train幾次epochs，設大一點沒關係，我們有eary_stop 機制
#display_step = 100  # 你可以用這個去建一個規則去看train的正確率，但本專案先不用
validation_step =50 # 當50步時去算一次驗證資料的正確率

checkpoint_dir = 'save/'


In [21]:
# Training cycle
total_batch = train_size // batch_size

max_acc = 0. # Save the maximum accuracy value for validation data
early_stop_limit = 0 # 紀錄early_stop的值
isTrain =True # 用來跳脫兩個迴圈

init = tf.global_variables_initializer()
## Returns an Op that initializes all local variables.
init_l = tf.local_variables_initializer()
saver = tf.train.Saver()  #call save function

In [22]:
with tf.Session(config=config) as sess:
    sess.run(init)
    sess.run(init_l)
    
    ##可以自己選擇大概要幾個epoch，因為我們有early stop的方法，所以不怕一直train
    for epoch in range(training_epochs):
        if(isTrain):
            ##Random shuffling
            idx = np.random.permutation(train_size)
            train_data , train_label = X_train1[idx] , y_train1[idx]

            ##用批次的方式去訓練model
            for i in range(total_batch):
                # Compute the offset of the current minibatch in the data.
                offset = (i * batch_size) % (train_size)
                ## 批次的方式去訓練
                batch_xs = train_data[offset:(offset + batch_size), :]
                batch_ys = train_label[offset:(offset + batch_size)]
                _ , _loss =sess.run( [training_op,loss] , feed_dict={x : batch_xs  , y : batch_ys})
                # 每 n step時，model去看此時的驗證資料的正確率並印出來
                if i % validation_step == 0:
                    validation_accuracy = sess.run(accuracy,feed_dict={x:X_valid1 , y:y_valid1 })
                    print("Epoch:", '%04d,' % (epoch + 1),"batch_index %4d/%4d , validation accuracy %.5f" % (i, total_batch, validation_accuracy))
                    # 最大驗證正確率大於每 n step的驗證正確率時，early_stop_limit + 1
                    # 這裡也可以使用loss的技巧 or recall or precsion 去做early stop
                    if max_acc > validation_accuracy:
                        early_stop_limit+=1
                        # 自己可以去限制最大驗證正確率不再變n次時，就停止訓練
                        if early_stop_limit == 300 :
                            isTrain = False
                            print('early_stop...........')
                            break
                    
                    # 如果每validation_step算出來的validation_accuracy大於max_acc
                    # 則取代它並儲存一次結果
                    if validation_accuracy > max_acc:
                        max_acc = validation_accuracy
                        saver.save(sess, checkpoint_dir + 'Team07_HW2.ckpt')
                        print('Team07_HW2.ckpt-'+'complete-%04d-' % (epoch + 1) + "batch_index-%d" % i)
                        early_stop_limit = 0


Epoch: 0001, batch_index    0/ 560 , validation accuracy 0.26740
Team07_HW2.ckpt-complete-0001-batch_index-0
Epoch: 0001, batch_index   50/ 560 , validation accuracy 0.91360
Team07_HW2.ckpt-complete-0001-batch_index-50
Epoch: 0001, batch_index  100/ 560 , validation accuracy 0.94449
Team07_HW2.ckpt-complete-0001-batch_index-100
Epoch: 0001, batch_index  150/ 560 , validation accuracy 0.94957
Team07_HW2.ckpt-complete-0001-batch_index-150
Epoch: 0001, batch_index  200/ 560 , validation accuracy 0.95661
Team07_HW2.ckpt-complete-0001-batch_index-200
Epoch: 0001, batch_index  250/ 560 , validation accuracy 0.95895
Team07_HW2.ckpt-complete-0001-batch_index-250
Epoch: 0001, batch_index  300/ 560 , validation accuracy 0.96403
Team07_HW2.ckpt-complete-0001-batch_index-300
Epoch: 0001, batch_index  350/ 560 , validation accuracy 0.96521
Team07_HW2.ckpt-complete-0001-batch_index-350
Epoch: 0001, batch_index  400/ 560 , validation accuracy 0.96716
Team07_HW2.ckpt-complete-0001-batch_index-400
Epoc

Epoch: 0009, batch_index  300/ 560 , validation accuracy 0.98866
Epoch: 0009, batch_index  350/ 560 , validation accuracy 0.98944
Epoch: 0009, batch_index  400/ 560 , validation accuracy 0.99101
Epoch: 0009, batch_index  450/ 560 , validation accuracy 0.99062
Epoch: 0009, batch_index  500/ 560 , validation accuracy 0.99179
Team07_HW2.ckpt-complete-0009-batch_index-500
Epoch: 0009, batch_index  550/ 560 , validation accuracy 0.99179
Epoch: 0010, batch_index    0/ 560 , validation accuracy 0.99218
Team07_HW2.ckpt-complete-0010-batch_index-0
Epoch: 0010, batch_index   50/ 560 , validation accuracy 0.99101
Epoch: 0010, batch_index  100/ 560 , validation accuracy 0.98984
Epoch: 0010, batch_index  150/ 560 , validation accuracy 0.99101
Epoch: 0010, batch_index  200/ 560 , validation accuracy 0.99101
Epoch: 0010, batch_index  250/ 560 , validation accuracy 0.99101
Epoch: 0010, batch_index  300/ 560 , validation accuracy 0.99179
Epoch: 0010, batch_index  350/ 560 , validation accuracy 0.99023


Epoch: 0020, batch_index   50/ 560 , validation accuracy 0.99101
Epoch: 0020, batch_index  100/ 560 , validation accuracy 0.99218
Epoch: 0020, batch_index  150/ 560 , validation accuracy 0.99179
Epoch: 0020, batch_index  200/ 560 , validation accuracy 0.99062
Epoch: 0020, batch_index  250/ 560 , validation accuracy 0.99218
Epoch: 0020, batch_index  300/ 560 , validation accuracy 0.99062
Epoch: 0020, batch_index  350/ 560 , validation accuracy 0.99140
Epoch: 0020, batch_index  400/ 560 , validation accuracy 0.99101
Epoch: 0020, batch_index  450/ 560 , validation accuracy 0.99140
Epoch: 0020, batch_index  500/ 560 , validation accuracy 0.99140
Epoch: 0020, batch_index  550/ 560 , validation accuracy 0.99023
Epoch: 0021, batch_index    0/ 560 , validation accuracy 0.99062
Epoch: 0021, batch_index   50/ 560 , validation accuracy 0.99140
Epoch: 0021, batch_index  100/ 560 , validation accuracy 0.99101
Epoch: 0021, batch_index  150/ 560 , validation accuracy 0.99101
Epoch: 0021, batch_index 

Epoch: 0030, batch_index  500/ 560 , validation accuracy 0.99179
Epoch: 0030, batch_index  550/ 560 , validation accuracy 0.99218
Epoch: 0031, batch_index    0/ 560 , validation accuracy 0.99179
Epoch: 0031, batch_index   50/ 560 , validation accuracy 0.99257
Epoch: 0031, batch_index  100/ 560 , validation accuracy 0.99296
Epoch: 0031, batch_index  150/ 560 , validation accuracy 0.99296
Epoch: 0031, batch_index  200/ 560 , validation accuracy 0.99257
Epoch: 0031, batch_index  250/ 560 , validation accuracy 0.99296
Epoch: 0031, batch_index  300/ 560 , validation accuracy 0.99296
Epoch: 0031, batch_index  350/ 560 , validation accuracy 0.99296
Epoch: 0031, batch_index  400/ 560 , validation accuracy 0.99335
Team07_HW2.ckpt-complete-0031-batch_index-400
Epoch: 0031, batch_index  450/ 560 , validation accuracy 0.99257
Epoch: 0031, batch_index  500/ 560 , validation accuracy 0.99296
Epoch: 0031, batch_index  550/ 560 , validation accuracy 0.99257
Epoch: 0032, batch_index    0/ 560 , validat

Epoch: 0041, batch_index  250/ 560 , validation accuracy 0.99257
Epoch: 0041, batch_index  300/ 560 , validation accuracy 0.99218
Epoch: 0041, batch_index  350/ 560 , validation accuracy 0.99296
Epoch: 0041, batch_index  400/ 560 , validation accuracy 0.99296
Epoch: 0041, batch_index  450/ 560 , validation accuracy 0.99296
Epoch: 0041, batch_index  500/ 560 , validation accuracy 0.99218
Epoch: 0041, batch_index  550/ 560 , validation accuracy 0.99257
Epoch: 0042, batch_index    0/ 560 , validation accuracy 0.99257
Epoch: 0042, batch_index   50/ 560 , validation accuracy 0.99179
Epoch: 0042, batch_index  100/ 560 , validation accuracy 0.99296
Epoch: 0042, batch_index  150/ 560 , validation accuracy 0.99218
Epoch: 0042, batch_index  200/ 560 , validation accuracy 0.99218
Epoch: 0042, batch_index  250/ 560 , validation accuracy 0.99179
Epoch: 0042, batch_index  300/ 560 , validation accuracy 0.99179
Epoch: 0042, batch_index  350/ 560 , validation accuracy 0.99218
Epoch: 0042, batch_index 

Epoch: 0052, batch_index  100/ 560 , validation accuracy 0.99257
Epoch: 0052, batch_index  150/ 560 , validation accuracy 0.99257
Epoch: 0052, batch_index  200/ 560 , validation accuracy 0.99257
Epoch: 0052, batch_index  250/ 560 , validation accuracy 0.99179
Epoch: 0052, batch_index  300/ 560 , validation accuracy 0.99218
Epoch: 0052, batch_index  350/ 560 , validation accuracy 0.99218
Epoch: 0052, batch_index  400/ 560 , validation accuracy 0.99218
Epoch: 0052, batch_index  450/ 560 , validation accuracy 0.99179
Epoch: 0052, batch_index  500/ 560 , validation accuracy 0.99218
Epoch: 0052, batch_index  550/ 560 , validation accuracy 0.99218
Epoch: 0053, batch_index    0/ 560 , validation accuracy 0.99179
Epoch: 0053, batch_index   50/ 560 , validation accuracy 0.99218
Epoch: 0053, batch_index  100/ 560 , validation accuracy 0.99218
Epoch: 0053, batch_index  150/ 560 , validation accuracy 0.99179
Epoch: 0053, batch_index  200/ 560 , validation accuracy 0.99218
Epoch: 0053, batch_index 

In [23]:
##開啟剛剛early_stop的model
with tf.Session() as sess:
    sess.run(init_l)
    saver.restore(sess, 'save/Team07_HW2.ckpt')
    ##計算測試集的accuracy 、 precision 、 recall
    print('Acc_test :' , sess.run(accuracy,feed_dict={x: X_test1, y: y_test1}))
    print('Prec_value :' , sess.run(precision,feed_dict={x: X_test1, y: y_test1}))
    print('Recall_value :' , sess.run(recall,feed_dict={x: X_test1, y: y_test1}))
    
    ## model去測試測試集並將label的結果記錄下來 
    pre = sess.run(tf.argmax(logits, 1), feed_dict={x:X_test1})

## 列印出測試資料的報表
print(classification_report(y_test1.ravel(), pre))

Acc_test : 0.991633
Prec_value : 0.999278
Recall_value : 0.998557
             precision    recall  f1-score   support

          0       0.99      1.00      1.00       980
          1       1.00      1.00      1.00      1135
          2       0.98      0.98      0.98      1032
          3       0.99      0.99      0.99      1010
          4       0.99      0.99      0.99       982

avg / total       0.99      0.99      0.99      5139



In [24]:
#######################################################
#                                                     #
# 加入 Dropout  ## 當dropout rate = 0.5 時 約略提升正確率 #                                                  #
#                                                     #
#######################################################

In [25]:
# 5層的DNN結構
# 差在有加入tf.layers.dropout(hidden_units , rate)
# 其他部分都跟上面是一樣的

he_init=tf.contrib.layers.variance_scaling_initializer(factor=2.0,
    mode='FAN_IN',
    uniform=False,
    seed=None,
    dtype=tf.float32)

with tf.name_scope("DNN_with_droput1"):
    hidden1 = tf.layers.dense(x, h1_units, name="hidden1_", kernel_initializer= he_init,
                             use_bias=True,bias_initializer=he_init,activation=tf.nn.elu)
    hidden1= tf.layers.dropout(hidden1,rate=0.5)
    
    hidden2 = tf.layers.dense(hidden1, h2_units, name="hidden2_", kernel_initializer= he_init,
                             use_bias=True,bias_initializer=he_init,activation=tf.nn.elu)
    hidden2 = tf.layers.dropout(hidden2,rate =0.5)
    
    hidden3 = tf.layers.dense(hidden2, h3_units, name="hidden3_", kernel_initializer= he_init,
                             use_bias=True,bias_initializer=he_init,activation=tf.nn.elu)
    hidden3 = tf.layers.dropout(hidden3,rate =0.5)
    
    hidden4 = tf.layers.dense(hidden3, h4_units, name="hidden4_", kernel_initializer= he_init,
                             use_bias=True,bias_initializer=he_init,activation=tf.nn.elu)
    hidden4 = tf.layers.dropout(hidden4,rate=0.5)
    
    hidden5 = tf.layers.dense(hidden4, h5_units, name="hidden5_", kernel_initializer= he_init,
                             use_bias=True,bias_initializer=he_init,activation=tf.nn.elu)
    hidden5 = tf.layers.dropout(hidden5,rate=0.5)

    logitss = tf.layers.dense(hidden5, n_class, name="outputs_")



In [26]:
with tf.name_scope("loss_with_droupt"):
    d_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logitss)
    d_loss = tf.reduce_mean(d_entropy, name="loss")
    
with tf.name_scope("train_"):
    batch = tf.Variable(0)

    learning_rate = tf.train.exponential_decay(
            1e-4,  # Base learning rate.
            batch * batch_size,  # Current index into the dataset.
            train_size,  # Decay step.
            0.95,  # Decay rate.
            staircase=True)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(d_loss,global_step=batch)
    
tf.summary.scalar('learning_rate', learning_rate)


with tf.name_scope('accuracy'):
    correct = tf.equal(tf.argmax(logitss, 1), y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("precision"):
    _,precision = tf.metrics.precision(predictions = tf.argmax(logitss,1)
                                       , labels= y)
with tf.name_scope('recall'):
    _,recall = tf.metrics.recall(predictions = tf.argmax(logitss,1)
                                 , labels= y)

In [27]:
init = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
saver = tf.train.Saver()

In [28]:
# Training cycle
total_batch = train_size // batch_size
max_acc = 0. # Save the maximum accuracy value for validation data
early_stop_limit = 0 
isTrain =True 
saver = tf.train.Saver()

In [29]:
##可以自己選擇大概要幾個epoch，因為我們有early stop的方法，所以不怕一直train
with tf.Session(config=config) as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        if(isTrain):
            ##Random shuffling
            idx = np.random.permutation(train_size)
            train_data , train_label = X_train1[idx] , y_train1[idx]

            ##用批次的方式去訓練model
            for i in range(total_batch):
                # Compute the offset of the current minibatch in the data.
                offset = (i * batch_size) % (train_size)
                batch_xs = train_data[offset:(offset + batch_size), :]
                batch_ys = train_label[offset:(offset + batch_size)]
                acc_train, _ , _loss =sess.run( [accuracy,training_op,d_loss] , feed_dict={x :batch_xs  , y : batch_ys})
                
                # 每 n step時，model去看此時的驗證資料的正確率並印出來
                if i % validation_step == 0:
                    validation_accuracy = sess.run(accuracy,feed_dict={x:X_valid1 , y:y_valid1 })
                    print("Epoch:", '%04d,' % (epoch + 1),"batch_index %4d/%4d , validation accuracy %.5f" % (i, total_batch, validation_accuracy))
                    # 最大驗證正確率大於每 n step的驗證正確率時，early_stop_limit + 1
                    # 這裡也可以使用loss的技巧 or recall or precsion 去做early stop
                    if max_acc > validation_accuracy:
                        early_stop_limit+=1
                        # 自己可以去限制最大驗證正確率不再變n次時，就停止訓練
                        if early_stop_limit == 300 :
                            isTrain = False
                            print('early_stop...........')
                            break
                            
                    # 如果每validation_step算出來的validation_accuracy大於max_acc
                    # 則取代它並儲存一次結果
                    if validation_accuracy > max_acc:
                        max_acc = validation_accuracy
                        saver.save(sess, checkpoint_dir + 'Team07_HW2.ckpt')
                        print('Team07_HW2.ckpt-'+'complete-%04d-' % (epoch + 1) + "batch_index-%d" % i)
                        early_stop_limit = 0

Epoch: 0001, batch_index    0/ 560 , validation accuracy 0.20524
Team07_HW2.ckpt-complete-0001-batch_index-0
Epoch: 0001, batch_index   50/ 560 , validation accuracy 0.90618
Team07_HW2.ckpt-complete-0001-batch_index-50
Epoch: 0001, batch_index  100/ 560 , validation accuracy 0.93745
Team07_HW2.ckpt-complete-0001-batch_index-100
Epoch: 0001, batch_index  150/ 560 , validation accuracy 0.94840
Team07_HW2.ckpt-complete-0001-batch_index-150
Epoch: 0001, batch_index  200/ 560 , validation accuracy 0.95895
Team07_HW2.ckpt-complete-0001-batch_index-200
Epoch: 0001, batch_index  250/ 560 , validation accuracy 0.96052
Team07_HW2.ckpt-complete-0001-batch_index-250
Epoch: 0001, batch_index  300/ 560 , validation accuracy 0.96325
Team07_HW2.ckpt-complete-0001-batch_index-300
Epoch: 0001, batch_index  350/ 560 , validation accuracy 0.96482
Team07_HW2.ckpt-complete-0001-batch_index-350
Epoch: 0001, batch_index  400/ 560 , validation accuracy 0.96677
Team07_HW2.ckpt-complete-0001-batch_index-400
Epoc

Epoch: 0009, batch_index  500/ 560 , validation accuracy 0.98866
Epoch: 0009, batch_index  550/ 560 , validation accuracy 0.99023
Epoch: 0010, batch_index    0/ 560 , validation accuracy 0.99062
Team07_HW2.ckpt-complete-0010-batch_index-0
Epoch: 0010, batch_index   50/ 560 , validation accuracy 0.98788
Epoch: 0010, batch_index  100/ 560 , validation accuracy 0.98905
Epoch: 0010, batch_index  150/ 560 , validation accuracy 0.98866
Epoch: 0010, batch_index  200/ 560 , validation accuracy 0.98749
Epoch: 0010, batch_index  250/ 560 , validation accuracy 0.98984
Epoch: 0010, batch_index  300/ 560 , validation accuracy 0.98905
Epoch: 0010, batch_index  350/ 560 , validation accuracy 0.98827
Epoch: 0010, batch_index  400/ 560 , validation accuracy 0.98905
Epoch: 0010, batch_index  450/ 560 , validation accuracy 0.98827
Epoch: 0010, batch_index  500/ 560 , validation accuracy 0.98944
Epoch: 0010, batch_index  550/ 560 , validation accuracy 0.98944
Epoch: 0011, batch_index    0/ 560 , validatio

Epoch: 0020, batch_index   50/ 560 , validation accuracy 0.99023
Epoch: 0020, batch_index  100/ 560 , validation accuracy 0.99140
Team07_HW2.ckpt-complete-0020-batch_index-100
Epoch: 0020, batch_index  150/ 560 , validation accuracy 0.99101
Epoch: 0020, batch_index  200/ 560 , validation accuracy 0.99101
Epoch: 0020, batch_index  250/ 560 , validation accuracy 0.99101
Epoch: 0020, batch_index  300/ 560 , validation accuracy 0.99140
Epoch: 0020, batch_index  350/ 560 , validation accuracy 0.99140
Epoch: 0020, batch_index  400/ 560 , validation accuracy 0.99062
Epoch: 0020, batch_index  450/ 560 , validation accuracy 0.99062
Epoch: 0020, batch_index  500/ 560 , validation accuracy 0.99062
Epoch: 0020, batch_index  550/ 560 , validation accuracy 0.99101
Epoch: 0021, batch_index    0/ 560 , validation accuracy 0.99101
Epoch: 0021, batch_index   50/ 560 , validation accuracy 0.99140
Epoch: 0021, batch_index  100/ 560 , validation accuracy 0.99101
Epoch: 0021, batch_index  150/ 560 , validat

Epoch: 0030, batch_index  200/ 560 , validation accuracy 0.99179
Epoch: 0030, batch_index  250/ 560 , validation accuracy 0.99257
Epoch: 0030, batch_index  300/ 560 , validation accuracy 0.99140
Epoch: 0030, batch_index  350/ 560 , validation accuracy 0.99101
Epoch: 0030, batch_index  400/ 560 , validation accuracy 0.99179
Epoch: 0030, batch_index  450/ 560 , validation accuracy 0.99179
Epoch: 0030, batch_index  500/ 560 , validation accuracy 0.99179
Epoch: 0030, batch_index  550/ 560 , validation accuracy 0.99218
Epoch: 0031, batch_index    0/ 560 , validation accuracy 0.99218
Epoch: 0031, batch_index   50/ 560 , validation accuracy 0.99140
Epoch: 0031, batch_index  100/ 560 , validation accuracy 0.99179
Epoch: 0031, batch_index  150/ 560 , validation accuracy 0.99179
Epoch: 0031, batch_index  200/ 560 , validation accuracy 0.99179
Epoch: 0031, batch_index  250/ 560 , validation accuracy 0.99140
Epoch: 0031, batch_index  300/ 560 , validation accuracy 0.99179
Epoch: 0031, batch_index 

Epoch: 0040, batch_index  550/ 560 , validation accuracy 0.99179
Epoch: 0041, batch_index    0/ 560 , validation accuracy 0.99179
Epoch: 0041, batch_index   50/ 560 , validation accuracy 0.99218
Epoch: 0041, batch_index  100/ 560 , validation accuracy 0.99179
Epoch: 0041, batch_index  150/ 560 , validation accuracy 0.99218
Epoch: 0041, batch_index  200/ 560 , validation accuracy 0.99218
Epoch: 0041, batch_index  250/ 560 , validation accuracy 0.99218
Epoch: 0041, batch_index  300/ 560 , validation accuracy 0.99179
Epoch: 0041, batch_index  350/ 560 , validation accuracy 0.99218
Epoch: 0041, batch_index  400/ 560 , validation accuracy 0.99179
Epoch: 0041, batch_index  450/ 560 , validation accuracy 0.99179
Epoch: 0041, batch_index  500/ 560 , validation accuracy 0.99218
Epoch: 0041, batch_index  550/ 560 , validation accuracy 0.99218
Epoch: 0042, batch_index    0/ 560 , validation accuracy 0.99218
Epoch: 0042, batch_index   50/ 560 , validation accuracy 0.99218
Epoch: 0042, batch_index 

Epoch: 0051, batch_index  400/ 560 , validation accuracy 0.99218
Epoch: 0051, batch_index  450/ 560 , validation accuracy 0.99218
Epoch: 0051, batch_index  500/ 560 , validation accuracy 0.99218
Epoch: 0051, batch_index  550/ 560 , validation accuracy 0.99218
Epoch: 0052, batch_index    0/ 560 , validation accuracy 0.99218
Epoch: 0052, batch_index   50/ 560 , validation accuracy 0.99218
Epoch: 0052, batch_index  100/ 560 , validation accuracy 0.99179
Epoch: 0052, batch_index  150/ 560 , validation accuracy 0.99179
Epoch: 0052, batch_index  200/ 560 , validation accuracy 0.99179
Epoch: 0052, batch_index  250/ 560 , validation accuracy 0.99218
Epoch: 0052, batch_index  300/ 560 , validation accuracy 0.99218
Epoch: 0052, batch_index  350/ 560 , validation accuracy 0.99218
Epoch: 0052, batch_index  400/ 560 , validation accuracy 0.99179
Epoch: 0052, batch_index  450/ 560 , validation accuracy 0.99179
Epoch: 0052, batch_index  500/ 560 , validation accuracy 0.99179
Epoch: 0052, batch_index 

In [30]:
with tf.Session() as sess:
    sess.run(init_l)
    sess.run(init)
    ##開啟剛剛early_stop的model
    saver.restore(sess, 'save/Team07_HW2.ckpt')
    ##計算測試集的accuracy 、 precision 、 recall
    print('Acc_test :' , sess.run(accuracy,feed_dict={x: X_test1, y: y_test1}))
    print('Prec_value :' , sess.run(precision,feed_dict={x: X_test1, y: y_test1}))
    print('Recall_value :' , sess.run(recall,feed_dict={x: X_test1, y: y_test1}))
    
    ## model去測試測試集並將label的結果記錄下來 
    pre = sess.run(tf.argmax(logitss, 1), feed_dict={x:X_test1})

## 列印出測試資料的報表
print(classification_report(y_test1.ravel(), pre))

Acc_test : 0.993773
Prec_value : 0.998798
Recall_value : 0.999038
             precision    recall  f1-score   support

          0       1.00      0.99      1.00       980
          1       1.00      1.00      1.00      1135
          2       0.99      0.99      0.99      1032
          3       0.99      0.99      0.99      1010
          4       1.00      0.99      0.99       982

avg / total       0.99      0.99      0.99      5139



In [31]:
###################################
#                                 #
#     K-FOLD cross validation     #
#                                 #
###################################

In [32]:
### 額外程式，主要去打亂合併的資料，並做k-fold 的取資料
class CrossValidationFolds(object):
    
    def __init__(self, data, labels, num_folds, shuffle=True):
        self.data = data
        self.labels = labels
        self.num_folds = num_folds
        self.current_fold = 0
        
        # Shuffle Dataset
        if shuffle:
            perm = np.random.permutation(self.data.shape[0]) ##隨機打亂資料
            data = data[perm]
            labels = labels[perm]
    
    def split(self):
        current = self.current_fold
        size = int(self.data.shape[0]/self.num_folds) # 30596 / 5 一塊k的size大小
        
        index = np.arange(self.data.shape[0]) 
        ## lower_bound  和 upper_bound 去區分不同區塊部分的index
        ## 用 True 跟 False 去代表有符合的區塊 @@@K-1
        lower_bound = index >= current*size  
        upper_bound = index < (current + 1)*size
        
        ## 接下train後的部分，為validation  剩下的1 
        cv_region = lower_bound*upper_bound

        cv_data = self.data[cv_region]
        train_data = self.data[~cv_region]
        
        cv_labels = self.labels[cv_region]
        train_labels = self.labels[~cv_region]
        
        self.current_fold += 1 ## 丟回下一的fold
        return (train_data, train_labels), (cv_data, cv_labels)

In [33]:
#X_train1 和 X_vaild1 資料合併
#y_train1,y_valid1 資料合併
#目的是之後的打亂取k-1個當訓練
X_train = np.concatenate((X_train1,X_valid1),axis=0)
Y_label = np.concatenate((y_train1,y_valid1),axis=0)
train_size = len(X_train)
print(X_train.shape)
print(Y_label.shape)
FOLDS=5 ## k = 5

data = CrossValidationFolds(X_train, Y_label, FOLDS)
validation_step = 50

recalll_test = [] # 放每個第n-fold 時的recall值
precc_test = []   # 放每個第n-fold 時的精確值
accc_test = []    # 放每個第n-fold 時的正確率值

(30596, 784)
(30596,)


In [34]:
with tf.Session(config=config) as sess:
    # 做FOLFS次數
    for i in range(FOLDS):
        
        sess.run(init)
        sess.run(init_l)
        # 第n個Fold開使
        print('Current fold: {}\n'.format(data.current_fold+1))
        print()
        # 每個fold 產生的合再一起的四個訓練、一個驗證
        (train_input, train_target), (cv_input, cv_target) = data.split()
        
        # 因為有跳脫單次fold時的迴圈，當新的第n-fold時，要讓它進而去訓練
        isTrain = True 
        early_stop_limit = 0 
        max_acc = 0. 
        
        ##針對每個FOLD去做訓練
        for epoch in range(training_epochs):
            if(isTrain):
                # Random shuffling
                train_size = len(train_input)
                idx = np.random.permutation(train_size)
                train_input , train_target = train_input[idx] , train_target[idx]
                # 計算總batch
                total_batch = train_size // batch_size

                ##用批次的方式去訓練model
                for i in range(total_batch):
                    # Compute the offset of the current minibatch in the data.
                    offset = (i * batch_size) % (train_size)
                    batch_xs = train_input[offset:(offset + batch_size), :]
                    batch_ys = train_target[offset:(offset + batch_size)]
                    _ , _loss =sess.run( [training_op,d_loss] , feed_dict={x :batch_xs  , y : batch_ys})
                    # 每 n step時，model去看此時的驗證資料的正確率並印出來
                    if i % validation_step == 0:
                        #驗證資料用剛剛n-fold切出來的一個部分去算正確率
                        validation_accuracy = sess.run(accuracy,feed_dict={x:cv_input , y:cv_target })
                        print("Epoch:", '%04d,' % (epoch + 1),"batch_index %4d/%4d , validation accuracy %.5f" % (i, total_batch, validation_accuracy))
                        # 最大驗證正確率大於每 n step的驗證正確率時，early_stop_limit + 1
                        # 這裡也可以使用loss的技巧 or recall or precsion 去做early stop
                        # early stop 只在於 每個n-Fold
                        if max_acc > validation_accuracy:
                            early_stop_limit+=1
                            # 自己可以去限制最大驗證正確率不再變n次時，就停止訓練
                            if early_stop_limit == 300 :
                                isTrain = False
                                print('early_stop...........')
                                saver.restore(sess, 'save/Team07_HW2.ckpt')
                                # 計算第n個Fold，train好model後，去看測試集的accuracy 、 precision 、 recall
                                acc_test = sess.run(accuracy,feed_dict={x: X_test1, y: y_test1})
                                prec_test = sess.run(precision,feed_dict={x: X_test1, y: y_test1})
                                recall_test = sess.run(recall,feed_dict={x: X_test1, y: y_test1})
                                # 把三個數值接下來，之後在外面再印一次
                                accc_test.append(acc_test)
                                precc_test.append(prec_test)
                                recalll_test.append(recall_test)
                                print('Acc_test :' ,acc_test )
                                print('Prec_value :' ,prec_test )
                                print('Recall_value :' , recall_test)

                                break
                                
                       # 如果每validation_step算出來的validation_accuracy大於max_acc
                       # 則取代它並儲存一次結果
                        if validation_accuracy > max_acc:
                            max_acc = validation_accuracy
                            saver.save(sess, checkpoint_dir + 'Team07_HW2.ckpt')
                            print('Team07_HW2.ckpt-'+'complete-%04d-' % (epoch + 1) + "batch_index-%d" % i)
                            early_stop_limit = 0
                    


Current fold: 1


Epoch: 0001, batch_index    0/ 489 , validation accuracy 0.20248
Team07_HW2.ckpt-complete-0001-batch_index-0
Epoch: 0001, batch_index   50/ 489 , validation accuracy 0.90407
Team07_HW2.ckpt-complete-0001-batch_index-50
Epoch: 0001, batch_index  100/ 489 , validation accuracy 0.92940
Team07_HW2.ckpt-complete-0001-batch_index-100
Epoch: 0001, batch_index  150/ 489 , validation accuracy 0.94395
Team07_HW2.ckpt-complete-0001-batch_index-150
Epoch: 0001, batch_index  200/ 489 , validation accuracy 0.95016
Team07_HW2.ckpt-complete-0001-batch_index-200
Epoch: 0001, batch_index  250/ 489 , validation accuracy 0.95408
Team07_HW2.ckpt-complete-0001-batch_index-250
Epoch: 0001, batch_index  300/ 489 , validation accuracy 0.95702
Team07_HW2.ckpt-complete-0001-batch_index-300
Epoch: 0001, batch_index  350/ 489 , validation accuracy 0.95996
Team07_HW2.ckpt-complete-0001-batch_index-350
Epoch: 0001, batch_index  400/ 489 , validation accuracy 0.96372
Team07_HW2.ckpt-complete-0001-ba

Epoch: 0011, batch_index   50/ 489 , validation accuracy 0.98497
Epoch: 0011, batch_index  100/ 489 , validation accuracy 0.98513
Epoch: 0011, batch_index  150/ 489 , validation accuracy 0.98595
Team07_HW2.ckpt-complete-0011-batch_index-150
Epoch: 0011, batch_index  200/ 489 , validation accuracy 0.98595
Epoch: 0011, batch_index  250/ 489 , validation accuracy 0.98627
Team07_HW2.ckpt-complete-0011-batch_index-250
Epoch: 0011, batch_index  300/ 489 , validation accuracy 0.98546
Epoch: 0011, batch_index  350/ 489 , validation accuracy 0.98513
Epoch: 0011, batch_index  400/ 489 , validation accuracy 0.98627
Epoch: 0011, batch_index  450/ 489 , validation accuracy 0.98497
Epoch: 0012, batch_index    0/ 489 , validation accuracy 0.98627
Epoch: 0012, batch_index   50/ 489 , validation accuracy 0.98611
Epoch: 0012, batch_index  100/ 489 , validation accuracy 0.98660
Team07_HW2.ckpt-complete-0012-batch_index-100
Epoch: 0012, batch_index  150/ 489 , validation accuracy 0.98578
Epoch: 0012, batc

Epoch: 0023, batch_index  100/ 489 , validation accuracy 0.98823
Epoch: 0023, batch_index  150/ 489 , validation accuracy 0.98742
Epoch: 0023, batch_index  200/ 489 , validation accuracy 0.98791
Epoch: 0023, batch_index  250/ 489 , validation accuracy 0.98742
Epoch: 0023, batch_index  300/ 489 , validation accuracy 0.98807
Epoch: 0023, batch_index  350/ 489 , validation accuracy 0.98758
Epoch: 0023, batch_index  400/ 489 , validation accuracy 0.98742
Epoch: 0023, batch_index  450/ 489 , validation accuracy 0.98725
Epoch: 0024, batch_index    0/ 489 , validation accuracy 0.98742
Epoch: 0024, batch_index   50/ 489 , validation accuracy 0.98725
Epoch: 0024, batch_index  100/ 489 , validation accuracy 0.98742
Epoch: 0024, batch_index  150/ 489 , validation accuracy 0.98725
Epoch: 0024, batch_index  200/ 489 , validation accuracy 0.98758
Epoch: 0024, batch_index  250/ 489 , validation accuracy 0.98807
Epoch: 0024, batch_index  300/ 489 , validation accuracy 0.98840
Epoch: 0024, batch_index 

Epoch: 0035, batch_index  300/ 489 , validation accuracy 0.98840
Epoch: 0035, batch_index  350/ 489 , validation accuracy 0.98840
Epoch: 0035, batch_index  400/ 489 , validation accuracy 0.98840
Epoch: 0035, batch_index  450/ 489 , validation accuracy 0.98774
Epoch: 0036, batch_index    0/ 489 , validation accuracy 0.98840
Epoch: 0036, batch_index   50/ 489 , validation accuracy 0.98807
Epoch: 0036, batch_index  100/ 489 , validation accuracy 0.98791
Epoch: 0036, batch_index  150/ 489 , validation accuracy 0.98823
Epoch: 0036, batch_index  200/ 489 , validation accuracy 0.98807
Epoch: 0036, batch_index  250/ 489 , validation accuracy 0.98840
Epoch: 0036, batch_index  300/ 489 , validation accuracy 0.98856
Epoch: 0036, batch_index  350/ 489 , validation accuracy 0.98823
Epoch: 0036, batch_index  400/ 489 , validation accuracy 0.98840
Epoch: 0036, batch_index  450/ 489 , validation accuracy 0.98823
Epoch: 0037, batch_index    0/ 489 , validation accuracy 0.98856
Epoch: 0037, batch_index 

Epoch: 0048, batch_index  250/ 489 , validation accuracy 0.98856
Epoch: 0048, batch_index  300/ 489 , validation accuracy 0.98791
Epoch: 0048, batch_index  350/ 489 , validation accuracy 0.98807
Epoch: 0048, batch_index  400/ 489 , validation accuracy 0.98840
Epoch: 0048, batch_index  450/ 489 , validation accuracy 0.98840
Epoch: 0049, batch_index    0/ 489 , validation accuracy 0.98840
Epoch: 0049, batch_index   50/ 489 , validation accuracy 0.98856
Epoch: 0049, batch_index  100/ 489 , validation accuracy 0.98823
Epoch: 0049, batch_index  150/ 489 , validation accuracy 0.98791
Epoch: 0049, batch_index  200/ 489 , validation accuracy 0.98856
Epoch: 0049, batch_index  250/ 489 , validation accuracy 0.98791
Epoch: 0049, batch_index  300/ 489 , validation accuracy 0.98840
Epoch: 0049, batch_index  350/ 489 , validation accuracy 0.98807
Epoch: 0049, batch_index  400/ 489 , validation accuracy 0.98823
Epoch: 0049, batch_index  450/ 489 , validation accuracy 0.98823
Epoch: 0050, batch_index 

...

Epoch: 0020, batch_index    0/ 489 , validation accuracy 0.99118
Epoch: 0020, batch_index   50/ 489 , validation accuracy 0.99085
Epoch: 0020, batch_index  100/ 489 , validation accuracy 0.99134
Epoch: 0020, batch_index  150/ 489 , validation accuracy 0.99036
Epoch: 0020, batch_index  200/ 489 , validation accuracy 0.99101
Epoch: 0020, batch_index  250/ 489 , validation accuracy 0.99118
Epoch: 0020, batch_index  300/ 489 , validation accuracy 0.99036
Epoch: 0020, batch_index  350/ 489 , validation accuracy 0.99068
Epoch: 0020, batch_index  400/ 489 , validation accuracy 0.98987
Epoch: 0020, batch_index  450/ 489 , validation accuracy 0.99003
Epoch: 0021, batch_index    0/ 489 , validation accuracy 0.99052
Epoch: 0021, batch_index   50/ 489 , validation accuracy 0.99134
Epoch: 0021, batch_index  100/ 489 , validation accuracy 0.99118
Epoch: 0021, batch_index  150/ 489 , validation accuracy 0.99118
Epoch: 0021, batch_index  200/ 489 , validation accuracy 0.99150
Epoch: 0021, batch_index 

Epoch: 0032, batch_index  350/ 489 , validation accuracy 0.99134
Epoch: 0032, batch_index  400/ 489 , validation accuracy 0.99118
Epoch: 0032, batch_index  450/ 489 , validation accuracy 0.99068
Epoch: 0033, batch_index    0/ 489 , validation accuracy 0.99118
Epoch: 0033, batch_index   50/ 489 , validation accuracy 0.99118
Epoch: 0033, batch_index  100/ 489 , validation accuracy 0.99118
Epoch: 0033, batch_index  150/ 489 , validation accuracy 0.99134
Epoch: 0033, batch_index  200/ 489 , validation accuracy 0.99134
Epoch: 0033, batch_index  250/ 489 , validation accuracy 0.99101
Epoch: 0033, batch_index  300/ 489 , validation accuracy 0.99118
Epoch: 0033, batch_index  350/ 489 , validation accuracy 0.99118
Epoch: 0033, batch_index  400/ 489 , validation accuracy 0.99134
Epoch: 0033, batch_index  450/ 489 , validation accuracy 0.99085
Epoch: 0034, batch_index    0/ 489 , validation accuracy 0.99085
Epoch: 0034, batch_index   50/ 489 , validation accuracy 0.99118
Epoch: 0034, batch_index 

Epoch: 0045, batch_index  200/ 489 , validation accuracy 0.99118
Epoch: 0045, batch_index  250/ 489 , validation accuracy 0.99134
Epoch: 0045, batch_index  300/ 489 , validation accuracy 0.99118
Epoch: 0045, batch_index  350/ 489 , validation accuracy 0.99134
Epoch: 0045, batch_index  400/ 489 , validation accuracy 0.99167
Epoch: 0045, batch_index  450/ 489 , validation accuracy 0.99167
Epoch: 0046, batch_index    0/ 489 , validation accuracy 0.99134
Epoch: 0046, batch_index   50/ 489 , validation accuracy 0.99199
Team07_HW2.ckpt-complete-0046-batch_index-50
Epoch: 0046, batch_index  100/ 489 , validation accuracy 0.99150
Epoch: 0046, batch_index  150/ 489 , validation accuracy 0.99118
Epoch: 0046, batch_index  200/ 489 , validation accuracy 0.99150
Epoch: 0046, batch_index  250/ 489 , validation accuracy 0.99167
Epoch: 0046, batch_index  300/ 489 , validation accuracy 0.99216
Team07_HW2.ckpt-complete-0046-batch_index-300
Epoch: 0046, batch_index  350/ 489 , validation accuracy 0.99167

Epoch: 0058, batch_index    0/ 489 , validation accuracy 0.99167
Epoch: 0058, batch_index   50/ 489 , validation accuracy 0.99167
Epoch: 0058, batch_index  100/ 489 , validation accuracy 0.99167
Epoch: 0058, batch_index  150/ 489 , validation accuracy 0.99150
Epoch: 0058, batch_index  200/ 489 , validation accuracy 0.99167
Epoch: 0058, batch_index  250/ 489 , validation accuracy 0.99167
Epoch: 0058, batch_index  300/ 489 , validation accuracy 0.99199
Epoch: 0058, batch_index  350/ 489 , validation accuracy 0.99183
Epoch: 0058, batch_index  400/ 489 , validation accuracy 0.99199
Epoch: 0058, batch_index  450/ 489 , validation accuracy 0.99150
Epoch: 0059, batch_index    0/ 489 , validation accuracy 0.99150
Epoch: 0059, batch_index   50/ 489 , validation accuracy 0.99167
Epoch: 0059, batch_index  100/ 489 , validation accuracy 0.99183
Epoch: 0059, batch_index  150/ 489 , validation accuracy 0.99183
Epoch: 0059, batch_index  200/ 489 , validation accuracy 0.99150
Epoch: 0059, batch_index 

Epoch: 0070, batch_index  450/ 489 , validation accuracy 0.99199
Epoch: 0071, batch_index    0/ 489 , validation accuracy 0.99199
Epoch: 0071, batch_index   50/ 489 , validation accuracy 0.99199
Epoch: 0071, batch_index  100/ 489 , validation accuracy 0.99183
Epoch: 0071, batch_index  150/ 489 , validation accuracy 0.99183
Epoch: 0071, batch_index  200/ 489 , validation accuracy 0.99199
Epoch: 0071, batch_index  250/ 489 , validation accuracy 0.99167
Epoch: 0071, batch_index  300/ 489 , validation accuracy 0.99199
Epoch: 0071, batch_index  350/ 489 , validation accuracy 0.99183
Epoch: 0071, batch_index  400/ 489 , validation accuracy 0.99199
Epoch: 0071, batch_index  450/ 489 , validation accuracy 0.99183
Epoch: 0072, batch_index    0/ 489 , validation accuracy 0.99167
Epoch: 0072, batch_index   50/ 489 , validation accuracy 0.99183
Epoch: 0072, batch_index  100/ 489 , validation accuracy 0.99183
Epoch: 0072, batch_index  150/ 489 , validation accuracy 0.99183
Epoch: 0072, batch_index 

In [35]:
print(accc_test)
print(recalll_test)
print(precc_test)
print('1. 5-Fold平均正確率 : ',np.array(accc_test).mean() , 
      '2. 5-Fold平均精確率 : ',np.array(precc_test).mean() ,
      '3. 5-Fold平均Recall : ',np.array(recalll_test).mean())

[0.99280006, 0.99338388, 0.99104869, 0.99221635, 0.99280012]
[0.99879777, 0.99855733, 0.99879777, 0.99855733, 0.99879777]
[0.99855769, 0.99927813, 0.99855769, 0.99903774, 0.99879777]
1. 5-Fold平均正確率 :  0.99245 2. 5-Fold平均精確率 :  0.998846 3. 5-Fold平均Recall :  0.998702


In [36]:
#####################################
#                                   #
# Explain the training process: 30% #
#                                   #
#####################################

In [37]:
##########################################################################
#首先先定義好DNN的架構，5層hidden layer 及 每一層128units，最終要辨識5個類別     #
#是屬於one-hot。所以要訓練的參數如 : W1[784,128] W2[128,128] W3[128,128]     #
#W4 [128,128] W5[128,128] ，每一層的units是否被啟動，本功課是用elu去當激勵     #
#函數，而最後一層softmax layer在轉成每個class的機率，train出來的label會去跟     #
#原本的label比較算出loss並平均，我們訓練model是以最小化這個loss，所以會Backward  #
#propagation 去更新訓練的參數，這次的功課是用Adam optimization 去找到最佳的方法。#
#                                                                        #
#Ps . 本文也有用實作droput的功能，將每一的隱層隨機將一部分的units置為0，目的是減輕  #
#過度擬合的問題。                                                           #
#                                                                        #
#本專題也有用mimi-batch的方式feed給placeholder，這種做法可以減少計算量大小，且容易#
#找到局部最佳結果。                                                        #
#                                                                        #
#本專題採用early stop機制，可以避免overfitting， 條件可以自己去設定，本研究先採用當#
#最大驗證機率不再更動300次(每n步去算一次驗證正確率)                             #
#                                                                        #
##########################################################################